I want to minimize the cost involved in running a Qdrant vector database on Google Cloud.

**Why Google Cloud Run for a running a vector database?**

Running on Google Cloud Run offers the following advantages
 - Auto Scaling with load
 - We can keep the --min-instances to 0 to to stop running when there is no load. 

But the challenge is - storage on the Cloud Run containers is ephemeral. So what would be the common persistent storage for Qdrant?

Luckily, from gen2, Cloud Run supports mounting a cloud storage bucket as a volume, which can be accessible across the cloud run instances. 

So here are the gcloud command to setup create the storage bucket and to update/deploy the cloud run service. 
```bash
# set the project name and region
export PROJECT_ID=project_xyz
export REGION=asia-southeast1
export BUCKET_NAME=bucket_name

# switch to the project
gcloud config set project $PROJECT_ID

# create a cloud storage bucket
gsutil mb -l $REGION gs://$BUCKET_NAME


# deploy the cloud run service
gcloud beta run deploy qdrant --image qdrant/qdrant:latest \
--execution-environment gen2 \
--add-volume=name=qdrant_storage,type=cloud-storage,bucket=$BUCKET_NAME \
--add-volume-mount=volume=qdrant_storage,mount-path=/qdrant/storage \
--allow-unauthenticated --max-instances 1 \
--update-env-vars QDRANT__SERVICE__HTTP_PORT=8080,QDRANT__SERVICE__API_KEY=$QDRANT__SERVICE__API_KEY \
--memory 512Mi \
--region $REGION --project $PROJECT_ID

# to update the cloud run service
gcloud beta run services update qdrant \
--region asia-southeast1 --project $PROJECT_ID \
--memory 512Mi --max-instances 1 \
```

